### Dump Pandas DataFrame in PSQL
Step 1: Create Connection
<br>
Step 2: Create Cursor
<br>
Step 3: Actual SQL
<br>
Step 4: Commit
<br>
Step 5: Close Connection

# Psycopg: PostgreSQL Adapter for Python

Psycopg is the most popular PostgreSQL adapter used in Python. It works on the principle of the whole implementation of Python DB API 2.0 along with thread safety (the same connection is shared by multiple threads). It is designed to perform heavily multi-threaded applications that usually create and destroy lots of cursors and make a large number of simultaneous `INSERT` or `UPDATE` operations.

## Key Features

- **Thread Safety:** The same connection can be shared by multiple threads.
- **High Performance:** Suitable for heavily multi-threaded applications.
- **Client-side and Server-side Cursors:** Efficient data fetching and manipulation.
- **Asynchronous Communication and Notification:** Allows for non-blocking database interactions.
- **Unicode and Python 3 Friendly:** Fully supports Unicode and is compatible with Python 3.

Psycopg is an excellent choice for applications that require robust and efficient interaction with PostgreSQL databases.


In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
import pandas as pd
import json
import os
from dataclasses import dataclass

@dataclass
class DBCredentials:
    """
    A dataclass to store database credentials.
    
    Attributes:
        user (str): Username for the database.
        password (str): Password for the database.
        host (str): Host address of the database.
        port (int): Port number to connect to the database.
        dbname (str): Name of the database.
    """
    user: str
    password: str
    host: str
    port: int
    dbname: str

def load_credentials(json_file: str) -> DBCredentials:
    """
    Load database credentials from a JSON file.
    
    Args:
        json_file (str): Path to the JSON file containing the credentials.
    
    Returns:
        DBCredentials: An instance of the DBCredentials dataclass.
    """
    with open(json_file, 'r') as file:
        creds_dict = json.load(file)
    return DBCredentials(**creds_dict)

def check_database_exists(creds: DBCredentials) -> bool:
    """
    Check if a database already exists.
    
    Args:
        creds (DBCredentials): The database credentials.
    
    Returns:
        bool: True if the database exists, False otherwise.
    """
    conn = psycopg2.connect(
        host=creds.host,
        user=creds.user,
        password=creds.password,
        database='postgres'  # Connect to the default 'postgres' database to check
    )
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor = conn.cursor()
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname='{creds.dbname}';")
    exists = cursor.fetchone() is not None
    cursor.close()
    conn.close()
    return exists

def create_db_and_dump_data(creds: DBCredentials, table_name: str, data_file: str, if_exists: str = 'replace'):
    """
    Create a database (if it doesn't already exist) and dump data into a table.
    
    Args:
        creds (DBCredentials): The database credentials.
        table_name (str): The name of the table to create or append data to.
        data_file (str): Path to the data file (Excel or CSV) to be dumped into the table.
        if_exists (str): What to do if the table already exists. Options are 'replace' or 'append'.
    """
    try:
        # Check if the database already exists
        if not check_database_exists(creds):
            # Connect to the default 'postgres' database to create the new database
            default_conn = psycopg2.connect(
                host=creds.host,
                user=creds.user,
                password=creds.password,
                database='postgres'
            )
            default_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
            default_cursor = default_conn.cursor()
            default_cursor.execute(f"CREATE DATABASE {creds.dbname};")
            print(f"Database '{creds.dbname}' created successfully.")
            default_cursor.close()
            default_conn.close()
        else:
            print(f"Database '{creds.dbname}' already exists. Skipping creation.")

        # Connect to the target database using SQLAlchemy
        engine = create_engine(f'postgresql+psycopg2://{creds.user}:{creds.password}@{creds.host}/{creds.dbname}')

        # Determine the file extension and load the data accordingly
        file_extension = os.path.splitext(data_file)[1].lower()
        
        if file_extension == '.xlsx':
            df = pd.read_excel(data_file)
        elif file_extension == '.csv':
            df = pd.read_csv(data_file)
        else:
            raise ValueError("Unsupported file format. Please use an Excel (.xlsx) or CSV (.csv) file.")

        # Dump data into the table
        df.to_sql(table_name, engine, if_exists=if_exists, index=False)
        print(f"Data dumped into table '{table_name}' successfully.")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    # Specify the table name and path to the data file
    your_table_name = 'testCell'
    data_to_dump_file_path = r"C:\Users\anita\OneDrive\Desktop\Ad-Hocs\testCell.csv"

    # Load credentials from the JSON file
    creds = load_credentials(r"C:\Users\anita\OneDrive\Desktop\Ad-Hocs\db_credentials.json")

    # Create the database (if it doesn't exist) and dump data into the table
    create_db_and_dump_data(creds, your_table_name, data_to_dump_file_path, if_exists = 'replace')  # Change 'replace' to 'append' if needed


Database 'adhoc' already exists. Skipping creation.
Data dumped into table 'testCell' successfully.
